## Matchups Analysis - Part 1
This notebook is geared at quality assessments of the matchups runs.

In [ ]:
import numpy as np


def value_locate(refx, x):
    """
    VALUE_LOCATE locates the positions of given values within a
    reference array.  The reference array need not be regularly
    spaced.  This is useful for various searching, sorting and
    interpolation algorithms.
    The reference array should be a monotonically increasing or
    decreasing list of values which partition the real numbers.  A
    reference array of NBINS numbers partitions the real number line
    into NBINS+1 regions, like so:
    REF:           X[0]         X[1]   X[2] X[3]     X[NBINS-1]
        <----------|-------------|------|---|----...---|--------------->
    INDICES:  -1           0          1    2       3        NBINS-1
    VALUE_LOCATE returns which partition each of the VALUES falls
    into, according to the figure above.  For example, a value between
    X[1] and X[2] would return a value of 1.  Values below X[0] return
    -1, and above X[NBINS-1] return NBINS-1.  Thus, besides the value
    of -1, the returned INDICES refer to the nearest reference value
    to the left of the requested value.
    
    Example:
    >>> refx = [2, 4, 6, 8, 10]
    >>> x = [-1, 1, 2, 3, 5, 5, 5, 8, 12, 30]
    >>> print value_locate(refx, x)
    array([-1, -1,  0,  0,  1,  1,  1,  3,  4,  4])
    
    
    This implementation is likely no the most efficient one, as there is
    a loop over all x, which will in practice be long. As long as x is
    shorter than 1e6 or so elements, it should still be fast (~sec).
    
    
    """
    print ("TODO: check if refx is monotonically increasing.")
    
    refx = np.array(refx)
    x = np.array(x)
    loc = np.zeros(len(x), dtype='int')
    
    for i in range(len(x)):
        ix = x[i]
        ind = ((refx - ix) <= 0).nonzero()[0]
        if len(ind) == 0:
            loc[i] = -1
        else: loc[i] = ind[-1]
    
    return loc
    


if __name__ =="__main__":
    # test case
    refx = [2, 4, 6, 8, 10]
    x = [-1, 1, 2, 3, 5, 5, 5, 8, 12, 30]
    
    res = value_locate(refx, x)
    assert list(res) == [-1, -1,  0,  0,  1,  1,  1,  3,  4,  4]
    print ("Test(s) passed!")
    
    x= np.random.random(1e6)*20
    res = value_locate(refx, x)
    print ("Done!")

In [98]:
import pandas as pd
import pickle
import glob
import os
import netCDF4 as nc
import numpy as np

In [41]:
def ReturnL2BaseName(splitChar='_',ext='L2'):
    '''Generator yielding basename to search the matchups table'''
# Building the list of files processed
    l2fiDir = '/accounts/ekarakoy/disk02/UNCERTAINTIES/Monte-Carlo/Matchups/L2/AllFiles/'
    l2Ext = '*.%s' % ext
    searchPat = os.path.join(l2fiDir,l2Ext)
    l2FiGenList = glob.iglob(searchPat)
    for l2file in l2FiGenList:
        basename = l2file.split('_')[0].split('/')[-1]
        yield basename, l2file

In [284]:
def GetData(fn,bands=['412','443','490','510','555','670']):
    rrsDict = dict.fromkeys(bands)
    rrsUncDict = dict.fromkeys(bands)
    with nc.Dataset(fn) as ds:
        gpV = ds.groups['geophysical_data'].variables
        for band in bands:
            try:
                rrsDict[band] = gpV['Rrs_%s' % band ][:]
                rrsUncDict[band] = gpV['Rrs_unc_%s' % band][:]
            except KeyError:
                rrsDict, rrsUncDict = None,None
                break
    return rrsDict,rrsUncDict

In [243]:
def GetTarget(df,baseName):
    '''Generates dataframe indices relevant to a given L2 file.'''
    dfSub = df[df['file_name'].str.contains(baseName)]
    for idx in dfSub.index.values:
        yield idx
        
def GetUncLatLonRng(fn):
    '''Return lat/lon rng, from each file used in unc sim, as lat & lon tuples.'''
    with nc.Dataset(fn) as ds:
        naV = ds.groups['navigation_data'].variables
        latitude = naV['latitude'][:]
        longitude = naV['longitude'][:]
    latRng = (latitude.min(),latitude.max())
    lonRng = (longitude.min(),longitude.max())
    return latRng,lonRng,latitude,longitude

def CheckLocation(mUpLat,mUpLon,uncLatRng,uncLonRng):
    mLat_is_in_rng = False
    mLon_is_in_rng = False
    if mUpLat >= uncLatRng[0] and mUpLat <= uncLatRng[1]:
        mLat_is_in_rng = True
    if mUpLon >= uncLonRng[0] and mUpLon <= uncLonRng[1]:
        mLon_is_in_rng = True
    return mLat_is_in_rng, mLon_is_in_rng    

In [159]:
def GetCntrPxlRowCol(latArray,mUpLat,lonArray,mUpLon):
    latLonDist = abs(latArray - mUpLat) + abs(lonArray - mUpLon)
    i,j = np.unravel_index(latLonDist.argmin(),latLonDist.shape)
    boxValid = (np.array((i,j)) >1).all() and i < (uncLat.shape[0] - 2)  \
                and j < (uncLat.shape[1] - 2)
    return (i,j),boxValid

In [277]:
def GetStats(rrsDict,rrsUncDict,boxCntrRowCol):
    bands = rrsDict.keys()
    subRrsDict = dict.fromkeys(bands)
    subRrsUncDict = dict.fromkeys(bands)
    i,j = boxCntrRowCol
    boxXRng = [i - 2,i + 3]
    boxYRng = [j - 2,j + 3]
    for band in bands:
        #print(band)
        subRrs = rrsDict[band][boxXRng[0]:boxXRng[1],boxYRng[0]:boxYRng[1]]
        subRrsUnc = rrsUncDict[band][boxXRng[0]:boxXRng[1],boxYRng[0]:boxYRng[1]]
        try:
            if subRrs.compressed().size >= (subRrs.size / 2):
                subRrsDict[band] = subRrs.compressed().mean()
            else:
                subRrsDict[band] = np.nan
        except AttributeError:
            subRrsDict[band] = subRrs.mean()
        try:
            if subRrsUnc.compressed().size >= (subRrsUnc.size / 2):
                subRrsUncDict[band] = np.sqrt(np.sum(np.power(subRrsUnc.compressed(),2))) / \
                                                subRrsUnc.compressed().size
            else:
                subRrsUncDict[band] = np.nan
        except AttributeError:
            subRrsUncDict[band]
    return subRrsDict, subRrsUncDict

In [10]:
# Load pandas Matchups table
dfName = 'dfMatchUpSWF.pkl'
if os.path.exists(dfName):
    dfMatchups = pd.read_pickle(dfName)
else:
    print('File not found!!')

In [54]:
# get whether a particular index' lat AND lon is within interval: 
    # record idx, basename, uncLatRng, mUpInLatRng, uncLonRng, mUpInLonRng: 
    # => dict #1
# for indices with lat AND lon in range: 
    # record idx, pixel coords in corresponding file,  
    # whether box of 5x5 around pixel coord is possible,
    # and number of valid cells in the 5x5 box.
    # => dict #2
# for validated indices:
    # record idx, basename, unc_Rrs_vvv,  unc_Rrs_unc_vvv
    # note that unc_Rrs_unc_vvv (like unc_Rrs_vvv) is calculated from the 5x5 box 
    # surrounding the pixel coord in dict #2. As such unc_Rrs_unc_vvv is calculated by 
    # propagating the uncertainties of the cells in the 5x5 box.
    # => dict #3


In [230]:
dict3Labels

['index',
 'insituRrs_412',
 'seawifsRrs_412',
 'Rrs_412',
 'Rrs_unc_412',
 'insituRrs_443',
 'seawifsRrs_443',
 'Rrs_443',
 'Rrs_unc_443',
 'insituRrs_490',
 'seawifsRrs_490',
 'Rrs_490',
 'Rrs_unc_490',
 'insituRrs_510',
 'seawifsRrs_510',
 'Rrs_510',
 'Rrs_unc_510',
 'insituRrs_555',
 'seawifsRrs_555',
 'Rrs_555',
 'Rrs_unc_555',
 'insituRrs_670',
 'seawifsRrs_670',
 'Rrs_670',
 'Rrs_unc_670']

In [288]:
'''
dict1Labels = ['index','uncLatRng','mUpInLatRng','uncLonRng','mUpInLonRng','rowValid']
dict2Labels = ['index','boxCntrRowCol','boxValid']
dict3Label1 = ['index']
dict3Label2 = [insituRrs_412,Rrs_412,Rrs_unc_412,...,Rrs_vvv, Rrs_unc_vvv]
dict4Labels = ['basename': indices relevant to the basename]
'''
dict1Labels = ['index','uncLatRng','mUpInLatRng','uncLonRng','mUpInLonRng','rowValid']
dict2Labels = ['index','boxCntrRowCol','boxValid','numValCells']
dict3Labels = ['index']
bands = ['412','443','490','510','555','670']
for band in bands:
    dict3Labels.append('insitu_Rrs%s' % band)
    dict3Labels.append('seawifs_Rrs%s' % band)
    dict3Labels.append('Rrs_%s' % band)
    dict3Labels.append('Rrs_unc_%s' % band)
summaryLatIn, summaryLonIn, totalEntries = 0,0,0
dict1 = {dcl: [] for dcl in dict1Labels}
dict2 = {dcl: [] for dcl in dict2Labels}
dict3 = {dcl: [] for dcl in dict3Labels}
dict4 = {}
baseNameGen = ReturnL2BaseName()
# go through files
totalFiles = 0
totalRows = 0
for baseName,l2FileName in baseNameGen:
    totalFiles += 1
    dfIdcs = GetTarget(dfMatchups,baseName)
    dict4[baseName] = []
    # go through all rows corresponding to a given file 
    firstPass = True
    for dfIdx in dfIdcs:
        totalRows += 1
        # first task is to figure out whether a valid correspondence exists between
        # matchup data and the relevant l2 file...
        dict1['index'].append(dfIdx)
        mUpLat = dfMatchups.latitude[dfIdx]
        mUpLon = dfMatchups.longitude[dfIdx]
        uncLatRng, uncLonRng, uncLat, uncLon = GetUncLatLonRng(l2FileName)
        dict1['uncLatRng'].append(uncLatRng)
        dict1['uncLonRng'].append(uncLonRng)
        mUpInLatRng, mUpInLonRng = CheckLocation(mUpLat, mUpLon, uncLatRng, uncLonRng)
        dict1['mUpInLatRng'].append(mUpInLatRng)
        dict1['mUpInLonRng'].append(mUpInLonRng)
        rowValid = mUpInLatRng and mUpInLonRng
        dict1['rowValid'].append(rowValid)
        if rowValid:
            dict2['index'].append(dfIdx)
            dict4[baseName].append(dfIdx)
            # Get cell closest in lat/lon to corres. mUp lat/lon
            roColTpl,boxVal = GetCntrPxlRowCol(uncLat, mUpLat, uncLon, mUpLon)
            dict2['boxCntrRowCol'].append(roColTpl)
            dict2['boxValid'].append(boxVal)
            if boxVal:
                if firstPass:
                    # Load file data (once even if multiple matchups in the same scene)
                    firstPass = False
                    rrsDict,rrsUncDict = GetData(l2FileName)
                if not rrsDict or not rrsUncDict:
                    continue
                subRrsDict,subRrsUncDict = GetStats(rrsDict,rrsUncDict,
                                                    boxCntrRowCol=roColTpl)
                for band in bands:
                    dict3['insitu_Rrs%s' % band].append(dfMatchups.loc[dfIdx,'insitu_Rrs%s' % band])
                    dict3['seawifs_Rrs%s' % band].append(dfMatchups.loc[dfIdx,'seawifs_Rrs%s' % band])
                    dict3['Rrs_%s' % band].append(subRrsDict[band])
                    dict3['Rrs_unc_%s' % band].append(subRrsUncDict[band])
                
                dict3['index'].append(dfIdx)

In [289]:
dfDict3 = pd.DataFrame(dict3)

In [293]:
dfDict3.to_pickle('mUpComp.pkl')

In [292]:
pwd

'/accounts/ekarakoy/DEV-ALL/Matchups'

In [279]:
dfMatchups.columns

Index(['cruise_name', 'file_name', 'date_time', 'latitude', 'longitude',
       'seawifs_npixel', 'seawifs_tdiff', 'seawifs_solz', 'seawifs_senz',
       'seawifs_cv', 'seawifs_es_thresh', 'seawifs_windspeed', 'insitu_Rrs412',
       'insitu_Rrs443', 'insitu_Rrs490', 'insitu_Rrs510', 'insitu_Rrs555',
       'insitu_Rrs670', 'seawifs_Rrs412', 'seawifs_Rrs443', 'seawifs_Rrs490',
       'seawifs_Rrs510', 'seawifs_Rrs555', 'seawifs_Rrs670', 'L1_name'],
      dtype='object')

In [266]:
rrsDict,rrsUncDict = GetData(l2FileName)

In [250]:
roColTpl

(49, 50)

In [281]:
ds.close()

In [282]:
ds = nc.Dataset(l2FileName)
gpv = ds.groups['geophysical_data'].variables

In [283]:
gpv.keys()

odict_keys(['Lt_412', 'Lt_443', 'Lt_490', 'Lt_510', 'Lt_555', 'Lt_670', 'Lt_765', 'Lt_865', 'solz', 'sola', 'senz', 'sena', 'Rrs_412', 'Rrs_443', 'Rrs_490', 'Rrs_510', 'Rrs_555', 'Rrs_670', 'Rrs_765', 'Rrs_865', 'chlor_a', 'a_412_qaa', 'a_443_qaa', 'a_490_qaa', 'a_510_qaa', 'a_555_qaa', 'a_670_qaa', 'a_765_qaa', 'a_865_qaa', 'bb_412_qaa', 'bb_443_qaa', 'bb_490_qaa', 'bb_510_qaa', 'bb_555_qaa', 'bb_670_qaa', 'bb_765_qaa', 'bb_865_qaa', 'aot_865', 'owtd', 'l2_flags'])

In [276]:
rrs

<class 'netCDF4._netCDF4.Variable'>
int16 Rrs_510(number_of_lines, pixels_per_line)
    long_name: Remote sensing reflectance at 510 nm
    scale_factor: 2e-06
    add_offset: 0.05
    units: sr^-1
    standard_name: surface_ratio_of_upwelling_radiance_emerging_from_sea_water_to_downwelling_radiative_flux_in_air
    _FillValue: -32767
    valid_min: -30000
    valid_max: 25000
    solar_irradiance: 1925.61
path = /geophysical_data
unlimited dimensions: 
current shape = (101, 101)
filling on

In [262]:
GetStats(rrsDict,rrsUncDict,roColTpl)

510


AttributeError: 'numpy.ndarray' object has no attribute 'compressed'

In [267]:
bands = rrsDict.keys()
subRrsDict = dict.fromkeys(bands)
subRrsUncDict = dict.fromkeys(bands)
i,j = roColTpl
boxXRng = [i - 2,i + 3]
boxYRng = [j - 2,j + 3]
band='510'
subRrs = rrsDict[band][boxXRng[0]:boxXRng[1],boxYRng[0]:boxYRng[1]]
subRrsUnc = rrsUncDict[band][boxXRng[0]:boxXRng[1],boxYRng[0]:boxYRng[1]]

In [270]:
dict3

{'Rrs_412': 0.020716321,
 'Rrs_443': 0.015005601,
 'Rrs_490': 0.0073612011,
 'Rrs_510': 0.0034284007,
 'Rrs_555': 0.0012360811,
 'Rrs_670': 0.00010368079,
 'Rrs_unc_412': 2.8225528076291085e-05,
 'Rrs_unc_443': 2.3723826743662357e-05,
 'Rrs_unc_490': 2.5629224255681036e-05,
 'Rrs_unc_510': 2.3024172987788917e-05,
 'Rrs_unc_555': 1.7366849351674319e-05,
 'Rrs_unc_670': 6.1589846154674889e-06,
 'index': [2378,
  1757,
  3861,
  2446,
  2447,
  1986,
  2428,
  1978,
  1799,
  3094,
  2059,
  3126,
  3127,
  3681,
  3682,
  1764,
  1765,
  1766,
  2055,
  2443,
  1323,
  2601,
  2069,
  1800,
  3866,
  2082,
  1780,
  1983,
  1320,
  3620,
  3621,
  3622,
  3623,
  3624,
  3625,
  3626,
  2373,
  1974,
  1335,
  2429,
  2070,
  2245,
  2377,
  1321,
  921,
  2114,
  2243,
  2425,
  1339,
  2396,
  3504,
  2071,
  1336,
  1980,
  2442,
  1775,
  2596,
  3139,
  3140,
  3141,
  3142,
  3143,
  3851,
  1801,
  2064,
  2394,
  3659,
  3660,
  3661,
  99,
  1314,
  1774,
  1778,
  1779,
  1956,

In [204]:
boxLims = [rowcol[0]-2,rowcol[0]+3,rowcol[1]-2,rowcol[1]+3]
abox=rrsDict['510'][boxLims[0]:boxLims[1],boxLims[2]:boxLims[3]]
bbox=rrsUncDict['510'][boxLims[0]:boxLims[1],boxLims[2]:boxLims[3]]

In [212]:
np.sqrt(np.sum(np.power(bbox.compressed(),2)))/bbox.compressed().size

1.3185633579269051e-05

In [218]:
subRrs,subRrsUnc=GetStats(rrsDict,rrsUncDict,rowcol)

In [224]:
for band in subRrs.keys():
    print('%s:Rrs=%.2e;Rrs_unc=%.2e' % (band,subRrs[band],subRrsUnc[band]))

510:Rrs=3.83e-03;Rrs_unc=1.32e-05
490:Rrs=5.93e-03;Rrs_unc=1.46e-05
443:Rrs=7.61e-03;Rrs_unc=1.74e-05
412:Rrs=8.06e-03;Rrs_unc=2.19e-05
670:Rrs=2.15e-04;Rrs_unc=6.18e-06
555:Rrs=1.87e-03;Rrs_unc=1.03e-05


In [228]:
dfSub.insitu_Rrs510,dfSub.seawifs_Rrs510

(id
 2113    0.003539
 Name: insitu_Rrs510, dtype: float64, id
 2113    0.00386
 Name: seawifs_Rrs510, dtype: float64)

In [208]:
np.power(z,2)

array([1, 1, 1, 0, 1, 1, 4, 1, 4])

array([1, 1, 1, 0, 1, 1, 2, 1, 2])

In [193]:
a.size - a.mask.sum()

6978

In [190]:
b= a.compressed()
a.size

10201

In [194]:
a.compressed().size

6978

In [185]:
a.dtype,b.dtype

(dtype('float32'), dtype('float32'))

In [95]:
uncLonRng

(-122.01864, -120.49246)

In [109]:
uncLatRng, uncLonRng,uncLat,uncLon = GetUncLatLonRng(l2FileName)

In [160]:
rowcol,val = GetCntrPxlRowCol(uncLat,mUpLat,uncLon,mUpLon)

In [161]:
rowcol

(29, 71)

In [162]:
val

True

In [116]:
a=np.array(rowcol)

In [154]:
(np.array(rowcol) >1).all() and rowcol[0] < uncLat.shape[0] - 2 \
and rowcol[1] < uncLat.shape[1] - 2

True

In [155]:
uncLat.shape[0] - 2

99

(29, 71, 1, 1)

In [89]:
for idx in dfIdcs:
    print(dfMatchups.ix[idx,'latitude'])
GetCntrPxlRowCol()

In [61]:
'''
dict1Labels = ['index','uncLatRng','mUpInLatRng','uncLonRng','mUpInLonRng']
dict2Labels = ['index','basename','boxCntrRowCol','boxValid','numValCells']
dict3Labels = ['index','basename',Rrs_412,Rrs_unc_412,...,Rrs_vvv, Rrs_unc_vvv]
'''

for idx in dfex.index.values:
    print(dfex.loc[idx,:])

cruise_name                                                   s000620w
file_name            S2000172164529.L2_MLAC.R0000021386_43N_43N_68W...
date_time                                          2000/06/20 14:30:00
latitude                                                        43.753
longitude                                                      -66.598
seawifs_npixel                                                     100
seawifs_tdiff                                                     9000
seawifs_solz                                                    21.371
seawifs_senz                                                    22.126
seawifs_cv                                                      0.0325
seawifs_es_thresh                                                  3.5
seawifs_windspeed                                                4.933
insitu_Rrs412                                               0.00248826
insitu_Rrs443                                               0.00247619
insitu

In [38]:
pickle.dump(mainDict,open('matchupsQADict_1.pkl','wb'))

In [51]:
mainDict

{'basename': ['S1999293204435',
  'S1999046171443',
  'S1998314160028',
  'S1997281102712',
  'S1997284110316',
  'S1999226220939',
  'S1998178131558',
  'S1998270224215',
  'S1998031221920',
  'S1998101164920',
  'S1998070200518',
  'S1999177170758',
  'S1998093172710',
  'S1998190171715',
  'S1997293192729',
  'S1998242095953',
  'S1998160130056',
  'S1999043164019',
  'S1998262200102',
  'S1998033220958',
  'S1999196162309',
  'S1998104204237',
  'S1998327172230',
  'S1999047225905',
  'S1998143101309',
  'S1999188170252',
  'S1999022205436',
  'S1998075221010',
  'S1997270134451',
  'S1998197155521',
  'S1998263204556',
  'S1999284204006',
  'S1998146104833',
  'S1997331000608',
  'S1999228202047',
  'S1999276194102',
  'S1998176114617',
  'S1999271110133',
  'S1998252023801',
  'S1998279161421',
  'S1998267202734',
  'S1999255122052',
  'S1998290224900',
  'S1998235112130',
  'S1998301174653',
  'S1999187161821',
  'S1998267170934',
  'S1999056162710',
  'S1998034225446',
  'S1997